# Especializar um Modelo Pré-treinado do Hugging Face

Este notebook apresenta um roteiro que detalha os passos necessários para utilizar e especializar modelos de linguagem pré-treinados para a tarefa de sumarização de textos em português, garantindo a instalação e configuração adequadas das bibliotecas necessárias, bem como a seleção e utilização de modelos apropriados para o idioma.

### 1. Instalação de dependências
Execute o seguinte bloco para instalar e atualizar as dependências. Em seguida reinicie o kernel antes de executar os demais blocos.

In [1]:
%%capture
!pip install transformers datasets torch
!pip install sentencepiece
!pip install tf-keras
!pip install transformers[torch] -U
!pip install accelerate -U
!pip install transformers datasets torch accelerate -U

### 2. Carregamento do Corpus e Pré-processamento

In [2]:
from datasets import load_dataset, Dataset

# Carrega o corpus do arquivo gerado pelo outro notebook deste repositório.
dataset = load_dataset('csv', data_files='combined_corpus.csv')

In [3]:
from datasets import DatasetDict

# Divide o dataset em 90% para treino e 10% para validação
train_test_split = dataset['train'].train_test_split(test_size=0.1)
dataset = DatasetDict({
    'train': train_test_split['train'],
    'validation': train_test_split['test']
})

print(dataset)


DatasetDict({
    train: Dataset({
        features: ['id', 'texto', 'summary'],
        num_rows: 82
    })
    validation: Dataset({
        features: ['id', 'texto', 'summary'],
        num_rows: 10
    })
})


### 3. Escolha do Modelo Pré-Treinado

In [4]:
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM

model_name = "rhaymison/flan-t5-portuguese-small-summarization"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForSeq2SeqLM.from_pretrained(model_name)

### 4. Tokenização e Preparação dos Dados

In [5]:
def preprocess_function(examples):
    inputs = [doc for doc in examples['texto']]
    model_inputs = tokenizer(inputs, max_length=512, truncation=True)

    with tokenizer.as_target_tokenizer():
        labels = tokenizer(examples['summary'], max_length=128, truncation=True)

    model_inputs['labels'] = labels['input_ids']
    return model_inputs

tokenized_datasets = dataset.map(preprocess_function, batched=True)


Map:   0%|          | 0/82 [00:00<?, ? examples/s]

c:\Users\jcf_s\AppData\Local\Programs\Python\Python312\Lib\site-packages\transformers\tokenization_utils_base.py:3946: UserWarning: `as_target_tokenizer` is deprecated and will be removed in v5 of Transformers. You can tokenize your labels by using the argument `text_target` of the regular `__call__` method (either in the same call as your input texts if you use the same keyword arguments, or in a separate call.
  warnings.warn(


Map:   0%|          | 0/10 [00:00<?, ? examples/s]

### 5. Configuração do Treinamento

In [8]:
from transformers import Seq2SeqTrainingArguments, Seq2SeqTrainer

training_args = Seq2SeqTrainingArguments(
    output_dir='./results',
    evaluation_strategy='epoch',
    learning_rate=5e-5,
    per_device_train_batch_size=8,
    per_device_eval_batch_size=8,
    weight_decay=0.01,
    save_total_limit=3,
    num_train_epochs=5,
    predict_with_generate=True
)

trainer = Seq2SeqTrainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_datasets['train'],
    eval_dataset=tokenized_datasets['validation'],
    tokenizer=tokenizer
)

### 6. Treinamento do Modelo

In [9]:
trainer.train()

  0%|          | 0/55 [00:00<?, ?it/s]

  0%|          | 0/2 [00:00<?, ?it/s]

{'eval_loss': 0.9531230926513672, 'eval_runtime': 6.3125, 'eval_samples_per_second': 1.584, 'eval_steps_per_second': 0.317, 'epoch': 1.0}


  0%|          | 0/2 [00:00<?, ?it/s]

{'eval_loss': 0.8640025854110718, 'eval_runtime': 6.3321, 'eval_samples_per_second': 1.579, 'eval_steps_per_second': 0.316, 'epoch': 2.0}


...

  0%|          | 0/2 [00:00<?, ?it/s]

{'eval_loss': 0.7858659029006958, 'eval_runtime': 6.4593, 'eval_samples_per_second': 1.548, 'eval_steps_per_second': 0.31, 'epoch': 4.0}


  0%|          | 0/2 [00:00<?, ?it/s]

{'eval_loss': 0.7780861854553223, 'eval_runtime': 6.3431, 'eval_samples_per_second': 1.577, 'eval_steps_per_second': 0.315, 'epoch': 5.0}
{'train_runtime': 1074.5773, 'train_samples_per_second': 0.382, 'train_steps_per_second': 0.051, 'train_loss': 1.1592186667702415, 'epoch': 5.0}


TrainOutput(global_step=55, training_loss=1.1592186667702415, metrics={'train_runtime': 1074.5773, 'train_samples_per_second': 0.382, 'train_steps_per_second': 0.051, 'total_flos': 76215106928640.0, 'train_loss': 1.1592186667702415, 'epoch': 5.0})

### 7. Avaliação e Ajustes Finais

In [10]:
from pprint import pprint

results = trainer.evaluate()

pprint(results, indent=2)

  0%|          | 0/2 [00:00<?, ?it/s]

{ 'epoch': 5.0,
  'eval_loss': 0.7780861854553223,
  'eval_runtime': 4.7624,
  'eval_samples_per_second': 2.1,
  'eval_steps_per_second': 0.42}


### 8. Registrando o Modelo

In [11]:
model.save_pretrained('./final_model')
tokenizer.save_pretrained('./final_model')

('./final_model\\tokenizer_config.json',
 './final_model\\special_tokens_map.json',
 './final_model\\tokenizer.json')

### 9. Uso do Modelo

In [22]:
from transformers import pipeline

# Carrega o modelo treinado
summarizer = pipeline('summarization', model='./final_model', tokenizer='./final_model')

# Caminho para o arquivo de teste
test_file_path = 'input-test.txt'

# Lê todo o conteúdo do arquivo de teste
with open(test_file_path, 'r', encoding='utf-8') as file:
    test_text = file.read().strip()

# Define o limite máximo de tokens na saída
max_length_output = 512  # Ajuste esse valor conforme necessário
min_length_output = 100  # Ajuste esse valor conforme necessário

# Resume o texto do arquivo
summary = summarizer("sumarize:" + test_text, max_length=max_length_output, min_length=min_length_output, do_sample=False)[0]['summary_text']

# Exibir o resumo
print("Resumo do texto:")
print(summary)

Token indices sequence length is longer than the specified maximum sequence length for this model (5658 > 512). Running this sequence through the model will result in indexing errors


Resumo do texto:
O Plano Estratégico Setorial 2024-2027 é aprovado pela Resoluço ConSUG/MD no 6, de 11 de dezembro de 2019, com os objetivos de defesa e a integridade internacionais em açes de empregadas para o desenvolvimento do Plano Plurianual 2024/2027, como apenas especialistas, apontam o que acontecerá aos riscos estratégicios. Os resultados so ocorrências após o sistema educacional, o controle e de adversos para a infraestrutura e as atividades fsicos do Brasil e dos EUA. A avaliaça ainda estabelece o plano de enfrentamento na cidadania, e que esforço no pode ser enfatido para efetividade a pessoal combatente e ataque a meio ambiental.


#### Teste 1
Parâmetros:
```py
model_name = "rhaymison/flan-t5-portuguese-small-summarization"
training_args = Seq2SeqTrainingArguments(
    output_dir='./results',
    evaluation_strategy='epoch',
    learning_rate=2e-5,
    per_device_train_batch_size=8,
    per_device_eval_batch_size=8,
    weight_decay=0.01,
    save_total_limit=3,
    num_train_epochs=3,
    predict_with_generate=True
)
```
Resultado:

Setor de Defesa é aprovado pela Resoluço ConSUG/MD no 6, de 11 de dezembro de 2020, e especialistas apontam que o Plano Estratégico Setorial 2024-2027 está alinhado com a cadeia de valor setorial, adequadamente preparadas e fortalecimento dos objetivos nacionais, como a primeira esforço da cidadania. Resultado de estudos e debates sobre temas ligados ao enfrentamento nacional e defeso no significa exclusividade para o desenvolvimento


#### Teste 2
Parâmetros:
```py
model_name = "rhaymison/flan-t5-portuguese-small-summarization"
training_args = Seq2SeqTrainingArguments(
    output_dir='./results',
    evaluation_strategy='steps',
    learning_rate=5e-5,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,
    weight_decay=0.01,
    save_total_limit=3,
    num_train_epochs=5,
    predict_with_generate=True
)
```

Resultado:

Conheça o Plano Estratégico Setorial 2024-2027 com o PESD 2020-2027, aprovado pela Resoluço ConSUG/MD no 6, de 11 de dezembro de 2019, apresenta a identidade estratégica do Setor de Defesa, como se segue em vigor, e esforço nacional, na garantia da lei e da ordem; entenda os objetivos para o desenvolvimento de recursos. Governo tem ainda especialistas que aconteceram o plano após a primeira açes de efetividade ao sistema

#### Teste 3

Parâmetros:
```py
model_name = "rhaymison/flan-t5-portuguese-small-summarization"
training_args = Seq2SeqTrainingArguments(
    output_dir='./results',
    evaluation_strategy='epoch',
    learning_rate=5e-5,
    per_device_train_batch_size=8,
    per_device_eval_batch_size=8,
    weight_decay=0.01,
    save_total_limit=3,
    num_train_epochs=5,
    predict_with_generate=True
)
```

Resultado:

O Plano Estratégico Setorial 2024-2027 é aprovado pela Resoluço ConSUG/MD no 6, de 11 de dezembro de 2019, e o Plano estabelece a prioridade do Setor de Defesa para o desenvolvimento nacional, com os objetivos de serem empregados para a integraçes nacionais, o controle, a proteça, em apoio à poltica externa, para promover a confiança mtua, coeso ambiental e controles martimas. Os efeitos ainda no acontece

### Teste t5-portuguese-small-summarization

In [ ]:
from transformers import pipeline
summarization = pipeline("summarization", model="rhaymison/flan-t5-portuguese-small-summarization", tokenizer="rhaymison/flan-t5-portuguese-small-summarization")

test_file_path = 'input-test.txt'
with open(test_file_path, 'r', encoding='utf-8') as file:
    test_text = file.read().strip()

output =  summarization("sumarize:" + test_text)

print("Resumo do texto:")
print(output)

Setor de Defesa é aprovado pela Resoluço ConSUG/MD no 2, de 2019, após açes em vigor com os objetivos de recursos. Plano estratégico deve ser realizado pelo PES 2024-2027, e especialistas no so preparados para o desenvolvimento nacional, ainda como a esforço da cidadania.